In [3]:
import sqlite3

# This script lists all tables in a COLMAP database and their columns.
# Calculation of reprojection errors is based on the 'points3D' and 'images' tables is not yet possible. The value has to be calculated manually, 
# as it is not directly saved in the database.

# Path to the COLMAP database
db_path = '/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-07-30/A-1/colmap/database.db'

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to list all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in the database:", tables)

# For each table, list its columns
for table in tables:
    print(f"\nColumns in table {table[0]}:")
    cursor.execute(f"PRAGMA table_info({table[0]});")
    columns = cursor.fetchall()
    for column in columns:
        print(column)

conn.close()


Tables in the database: [('cameras',), ('sqlite_sequence',), ('images',), ('keypoints',), ('descriptors',), ('matches',), ('two_view_geometries',)]

Columns in table cameras:
(0, 'camera_id', 'INTEGER', 1, None, 1)
(1, 'model', 'INTEGER', 1, None, 0)
(2, 'width', 'INTEGER', 1, None, 0)
(3, 'height', 'INTEGER', 1, None, 0)
(4, 'params', 'BLOB', 0, None, 0)
(5, 'prior_focal_length', 'INTEGER', 1, None, 0)

Columns in table sqlite_sequence:
(0, 'name', '', 0, None, 0)
(1, 'seq', '', 0, None, 0)

Columns in table images:
(0, 'image_id', 'INTEGER', 1, None, 1)
(1, 'name', 'TEXT', 1, None, 0)
(2, 'camera_id', 'INTEGER', 1, None, 0)
(3, 'prior_qw', 'REAL', 0, None, 0)
(4, 'prior_qx', 'REAL', 0, None, 0)
(5, 'prior_qy', 'REAL', 0, None, 0)
(6, 'prior_qz', 'REAL', 0, None, 0)
(7, 'prior_tx', 'REAL', 0, None, 0)
(8, 'prior_ty', 'REAL', 0, None, 0)
(9, 'prior_tz', 'REAL', 0, None, 0)

Columns in table keypoints:
(0, 'image_id', 'INTEGER', 1, None, 1)
(1, 'rows', 'INTEGER', 1, None, 0)
(2, 'cols',

In [ ]:
# dummy code: 
import sqlite3
import numpy as np

# Path to the COLMAP database
db_path = '/path/to/colmap.db'

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Step 1: Get camera intrinsics from the 'cameras' table
def get_camera_intrinsics(cursor):
    cursor.execute("SELECT camera_id, model, width, height, params FROM cameras")
    camera_data = cursor.fetchall()
    
    intrinsics = {}
    for row in camera_data:
        camera_id = row[0]
        params = np.frombuffer(row[4], dtype=np.float64)  # Camera parameters as BLOB
        intrinsics[camera_id] = params[:3]  # Focal length and principal point
    return intrinsics

# Step 2: Get camera poses (extrinsics) from the 'images' table
def get_camera_poses(cursor):
    cursor.execute("SELECT image_id, prior_qw, prior_qx, prior_qy, prior_qz, prior_tx, prior_ty, prior_tz FROM images")
    poses_data = cursor.fetchall()
    
    poses = {}
    for row in poses_data:
        image_id = row[0]
        qw, qx, qy, qz = row[1:5]
        tx, ty, tz = row[5:8]
        poses[image_id] = {
            'quaternion': (qw, qx, qy, qz),
            'translation': (tx, ty, tz)
        }
    return poses

# Step 3: Get 2D keypoints from the 'keypoints' table
def get_keypoints(cursor):
    cursor.execute("SELECT image_id, rows, cols, data FROM keypoints")
    keypoints_data = cursor.fetchall()
    
    keypoints = {}
    for row in keypoints_data:
        image_id = row[0]
        data = np.frombuffer(row[3], dtype=np.float32).reshape(row[1], row[2])
        keypoints[image_id] = data
    return keypoints

# Helper function: Convert quaternion to rotation matrix
def quaternion_to_rotation_matrix(q):
    qw, qx, qy, qz = q
    R = np.array([
        [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]
    ])
    return R

# Step 4: Compute reprojection error
def compute_reprojection_error(intrinsics, poses, keypoints):
    errors = []
    
    for image_id, pose in poses.items():
        if image_id in keypoints:
            q = pose['quaternion']
            t = np.array(pose['translation'])
            R = quaternion_to_rotation_matrix(q)
            
            # Assume intrinsics are [fx, fy, cx, cy]
            if image_id in intrinsics:
                fx, fy, cx, cy = intrinsics[image_id][:4]
            else:
                continue  # Skip if no intrinsics found for this image
            
            for kp in keypoints[image_id]:
                # Simulated reprojection using 3D point (for example, a dummy point at [0, 0, 0] in 3D space)
                point_3d = np.array([0, 0, 0])  # Placeholder for 3D point (replace with actual 3D points from SfM)
                projected_point_2d = R @ point_3d + t
                
                # Project to 2D using camera intrinsics
                u = fx * (projected_point_2d[0] / projected_point_2d[2]) + cx
                v = fy * (projected_point_2d[1] / projected_point_2d[2]) + cy
                
                # Compare with keypoint
                reprojection_error = np.sqrt((u - kp[0])**2 + (v - kp[1])**2)
                errors.append(reprojection_error)
    
    if errors:
        return np.mean(errors)
    else:
        return None

# Extract data from database
camera_intrinsics = get_camera_intrinsics(cursor)
camera_poses = get_camera_poses(cursor)
keypoints = get_keypoints(cursor)

# Compute the mean reprojection error
mean_error = compute_reprojection_error(camera_intrinsics, camera_poses, keypoints)
if mean_error is not None:
    print(f"Mean Reprojection Error: {mean_error}")
else:
    print("No reprojection error could be calculated.")

# Close the database connection
conn.close()
